In [1]:
import tensorflow as tf
import numpy as np
import pickle
import itertools

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def get_model(shape):
    model = Sequential()
    model.add(LSTM(200, input_shape=shape))
    model.add(Dropout(0.3))
    model.add(Dense(shape[1]))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model

def train_model(model, X, y, val_data, epochs=20, batch_size=50):
    history = model.fit(X, y, validation_data=val_data, epochs=epochs, batch_size=batch_size)
    return history
    
def save(model, name):
    model_json = model.to_json()
    with open(name + '.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name + '.h5')

def load(name):
    # load json and create model
    json_file = open(name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name + ".h5")
    return model

In [3]:
def get_sonnets(filename):
    """ Returns entire txt file as string. """
    str = ""
    with open(filename) as file:
        for line in file:
            line = line.strip()
            if line.isdigit() == False and len(line) > 0:
                str += line.lower() + "\n"
    return str

In [4]:
file = 'data/shakespeare.txt'
sons = get_sonnets(file)

print ("# chars in txt: " + str(len(sons)))

# chars in txt: 93674


In [ ]:
chars = set(sons) # get set of all characters
print(chars)
print ("# unique chars: " + str(len(chars)))

# create mapping of characters to unique ids
dic_char_to_id = dict((c, i) for i, c in enumerate(chars))
dic_id_to_char = dict((i, c) for i, c in enumerate(chars))

{'r', '(', 'l', 'k', 'e', 'n', 'v', 'w', '?', 'd', ' ', '.', 'b', 'o', 'y', ';', "'", 'q', 't', 'x', 'u', 'm', 'j', '!', 'g', 'h', ')', ',', 'c', 's', ':', 'z', '-', 'i', 'f', 'a', 'p', '\n'}
# unique chars: 38


In [ ]:
# create input, output sequences

leng = 40
step_size = 2
seqs, nxt_char = [], []
# take all possible subsequences of 40 consecutive characters
for i in range(0, len(sons) - leng, step_size):
    seqs.append(sons[i: i + leng])
    nxt_char.append(sons[i + leng])
print("# seqs: ", len(seqs))

# seqs:  46817


In [ ]:
# convert seqs to one-hot encoding
X = np.zeros((len(seqs), leng, len(chars)), dtype=np.bool)
y = np.zeros((len(seqs), len(chars)), dtype=np.bool)

for i, seq in enumerate(seqs):
    for j, char in enumerate(seq):
        X[i, j, dic_char_to_id[char]] = 1
    y[i, dic_char_to_id[nxt_char[i]]] = 1

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = get_model((leng, len(chars)))

In [ ]:
history = train_model(model, x_train, y_train, (x_test, y_test))

Train on 35112 samples, validate on 11705 samples
Epoch 1/20
35112/35112 [==============================] - 186s 5ms/step - loss: 2.6742 - val_loss: 2.3762
Epoch 2/20
35112/35112 [==============================] - 150s 4ms/step - loss: 2.2509 - val_loss: 2.2011
Epoch 3/20
35112/35112 [==============================] - 137s 4ms/step - loss: 2.1037 - val_loss: 2.0806
Epoch 4/20
35112/35112 [==============================] - 103s 3ms/step - loss: 2.0111 - val_loss: 1.9893
Epoch 5/20
35112/35112 [==============================] - 77s 2ms/step - loss: 1.9443 - val_loss: 1.9473
Epoch 6/20
35112/35112 [==============================] - 89s 3ms/step - loss: 1.8858 - val_loss: 1.8989
Epoch 7/20
35112/35112 [==============================] - 100s 3ms/step - loss: 1.8367 - val_loss: 1.8761
Epoch 8/20
35112/35112 [==============================] - 111s 3ms/step - loss: 1.7892 - val_loss: 1.8437
Epoch 9/20
35112/35112 [==============================] - 108s 3ms/step - loss: 1.7473 - val_loss: 1.827

In [ ]:
save(model, 'models/Shakespeare_1LAYER_200units_20_50_0.3DROPOUT_0.25split')

#150 units, 200 units, epochs, batch_size, dropout

In [ ]:
from matplotlib import pyplot

# plot train and validation loss
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.savefig('visuals/Shakespeare_1LAYER_200units_20_50_0.3DROPOUT_0.25split_loss.png')
pyplot.show()

In [ ]:
min(history.history['val_loss']) # 1.7592611852973024

In [ ]:
history.history['val_loss'].index(min(history.history['val_loss'])) # 10

In [ ]:
with open('Shakespeare_1LAYER_200units_20_50_0.3DROPOUT_0.25split.pkl', 'wb') as f:
    pickle.dump(history, f)

In [ ]:
def sample(preds, temperature = 1.0):
    ''' Helper function to sample an index from
    a probability array. Taken from StackOverflow/
    open source code on GitHub. '''

    preds = np.asarray(preds).astype('float')
    preds = np.log(preds) / temperature
    preds[preds == np.inf] = 0

    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return np.argmax(np.random.multinomial(1, preds, 1))

In [ ]:
nchar = 800 # length of each line produced from preds
for temp in [1.5, 0.75, 0.25]:
    print( '\n' + '\n' + 'temperature = ' + str(temp) + '\n')
    
    line = 'shall i compare thee to a summer\'s day?\n'
    sentence = line
    
    for i in range(nchar):
        x = np.zeros((1, leng, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, dic_char_to_id[char]] = 1

        preds = model.predict(x, verbose=False)[0]
        nxt_char = dic_id_to_char[sample(preds, temp)]

        line += nxt_char
        sentence = sentence[1:] + nxt_char
        
    print(line)